In [1]:
import argparse
import cv2
import glob
import os
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from gfpgan import GFPGANer
from realesrgan import RealESRGANer
from realesrgan.archs.srvgg_arch import SRVGGNetCompact
import tensorflow as tf
import time
from skimage import exposure



ANIME = False

path = "./result10.png"
save_path = "./modified.png"

netscale = 4
model_path_face = "./realesrgan/GFPGANv1.3.pth"
dni_weight = None

tile = 0
title_pad = 10
pre_pad = 0
half = None
gpu_id = None
upscale = 3.5
upscale = 4
img_mode = 'RGBA'

if ANIME:
  # R-ESRGAN + Anime
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=6, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus_anime_6B.pth"
else:
  # R-ESRGAN
  model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
  model_path_x4 = "./realesrgan/RealESRGAN_x4plus.pth"

upsampler = RealESRGANer(
        scale=netscale,
        model_path=model_path_x4,
        dni_weight=dni_weight,
        model=model,
        tile=tile,
        tile_pad=title_pad,
        pre_pad=pre_pad,
        half= not half,
        gpu_id= gpu_id
)

face_enhancer = GFPGANer(
            model_path=model_path_face,
            upscale=upscale,
            arch='clean',
            channel_multiplier=2,
            bg_upsampler=upsampler)

def resize_image(data):
  resized_img = tf.image.resize(
  images=data,
  # size=[270, 480],     
  size=[512, 512], 
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img

def resize_image2(data):
  resized_img = tf.image.resize(
  images=data,
  size=[256, 256],
  method=tf.image.ResizeMethod.BILINEAR,
  preserve_aspect_ratio=False,
  antialias=True,
  )    
  return resized_img


c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Matheus\anaconda3\envs\Pyflow\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [2]:
img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

img = resize_image(img).numpy()
start = time.time() 
_, _, output = face_enhancer.enhance(img, has_aligned=False, only_center_face=False, paste_back=True)
# output = exposure.equalize_hist(output)*255
# output, _ = upsampler.enhance(img, outscale=upscale)

end = time.time()

print(end-start)
cv2.imwrite(save_path, output)

8.423465728759766


True